<a href="https://colab.research.google.com/github/AbdullahProgrammerX/machine-learning/blob/main/RandomSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Hyperparameter Optimization with RandomSearchCV

In [2]:
!pip install catboost
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.simplefilter(action='ignore', category=Warning)

df = pd.read_csv("/content/drive/MyDrive/Datasets/diabetes.csv")

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
rf_model = RandomForestClassifier(random_state=17)
rf_random_params = {"max_depth":np.random.randint(5, 50, 10),
                    "max_features": [3, 5, 7, "sqrt", "auto"],
                    "min_samples_split": np.random.randint(2, 50, 20),
                    "n_estimators":[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]}

In [4]:
rf_random = RandomizedSearchCV(estimator=rf_model,
                               param_distributions=rf_random_params,
                               n_iter=100,
                               cv=3,
                               verbose=True,
                               random_state=17,
                               n_jobs=-1)

In [5]:
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=17),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': array([32, 35, 17, 27, 28, 45, 30, 23, 33, 47]),
                                        'max_features': [3, 5, 7, 'sqrt',
                                                         'auto'],
                                        'min_samples_split': array([15, 13, 20,  9, 22, 31, 23, 25, 25, 29, 41, 13, 42, 25, 10, 19, 12,
       16, 28, 25]),
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=17, verbose=True)

In [6]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 12,
 'max_features': 'sqrt',
 'max_depth': 45}

In [7]:
rf_random_final = rf_model.set_params(**rf_random.best_params_, random_state=17).fit(X, y)

In [8]:
cv_result = cross_validate(rf_random_final, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
print(cv_result['test_accuracy'].mean())
print(cv_result['test_f1'].mean())
print(cv_result['test_roc_auc'].mean())

0.7722264663441134
0.639840152713347
0.8359056603773587
